In [1]:
import sys
sys.path.append('../../code/')

In [17]:
%load_ext autoreload
%autoreload 2

from libs import io
from libs.visuals import facetgrids
from libs.metrics import helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Available LLMs: (24): llama-3.3-8b llama-4-scout llama-4-mav gpt-oss-20b gpt-oss-120b qwen3-8b qwen3-14b qwen3-32b qwen3-30b-a3b-2507 qwen3-235b-a22b-2507 gemma-3-12b gemma-3-27b mistral-small-3.2-24b mistral-medium-3 llama-3.1-70b llama-3.3-70b llama-3.1-405b grok-4-fast deepseek-chat-v3.1 deepseek-r1-0528 gemini-2.5-flash gemini-2.5-flash-grounded gemini-2.5-pro gemini-2.5-pro-grounded


/home/espinl/.conda/envs/py311_llmscholar/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [45]:
# Example assumes you created df["metric_name"] before aggregation.
df_per_attempt = pd.DataFrame()
for metric in helpers.METRICS:
    fn = io.path_join('../../results/interventions/tables', f'per_attempt_{metric}.csv')
    df = io.read_csv(fn, index_col=0)
    df_per_attempt = pd.concat([df_per_attempt, df])
df_per_attempt.shape


(418096, 11)

In [46]:
df_per_attempt.head(5)

,model,grounded,temperature,date,time,task_name,task_param,metric,task_attempt,n_total,n_counts
0,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_citations_high,1.0,NaN,NaN,NaN
1,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_citations_low,0.0,NaN,NaN,NaN
2,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_diverse,1.0,NaN,NaN,NaN
3,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_ethnicity_asian,1.0,NaN,NaN,NaN
4,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_ethnicity_black,1.0,NaN,NaN,NaN


In [47]:
# Group: infrastructure (model_access, model_size, model_class)
# Subgroup: open/closed, S/M/L/XL, reasoning/non-reasoning
infra_class = ['model_access', 'model_size', 'model_class']
df_summary = pd.DataFrame()
for c in infra_class:
    tmp = df_per_attempt.groupby([c, 'metric_name'])['metric_value'].agg(['mean', 'std', 'count'])
    tmp.loc[:,'group'] = c.split('_')[1]
    tmp.loc[:,'ci_low'] = 0
    tmp.loc[:,'ci_high'] = 0
    tmp.reset_index(inplace=True)
    tmp.rename(columns={c:'subgroup'}, inplace=True)
    df_summary = pd.concat([df_summary, tmp])

df_summary.head(5)


KeyError: 'model_access'

In [30]:
# Group: infrastructure (model_access, model_size, model_class)
# Subgroup: open/closed, S/M/L/XL, reasoning/non-reasoning
# SubSubgroup: temperature

df_temperature = io.read_csv('../../results/interventions/tables/temperature_per_attempt.csv', low_memory=False, index_col=0)
infra_class = ['model_access', 'model_size', 'model_class']
df_summary = pd.DataFrame()
for c in infra_class:
    tmp = df.groupby([c, 'metric_name', 'temperature'])['metric_value'].agg(['mean', 'std', 'count'])
    tmp.loc[:,'group'] = c.split('_')[1]
    tmp.loc[:,'ci_low'] = 0
    tmp.loc[:,'ci_high'] = 0
    tmp.reset_index(inplace=True)
    tmp.rename(columns={c:'subgroup'}, inplace=True)
    df_summary = pd.concat([df_summary, tmp])

df_summary.head(5)


,subgroup,metric_name,temperature,mean,std,count,group,ci_low,ci_high
0,open,connectedness,0.00,0.124907,0.147803,4636,access,0,0
1,open,connectedness,0.25,0.079949,0.074704,4517,access,0,0
2,open,connectedness,0.50,0.139368,0.179114,3411,access,0,0
3,open,connectedness,0.75,0.086869,0.102728,1769,access,0,0
4,open,connectedness,1.00,0.159940,0.202291,1892,access,0,0


In [29]:
# Group: model
# Subgroup: task_name

df_summary = pd.DataFrame()

df_summary = df.groupby(['model', 'task_name'])['metric_value'].agg(['mean', 'std', 'count'])
df_summary.loc[:,'ci_low'] = 0
df_summary.loc[:,'ci_high'] = 0
df_summary.reset_index(inplace=True)
df_summary.head(5)


,model,task_name,mean,std,count,ci_low,ci_high
0,deepseek-chat-v3.1,epoch,0.528706,0.388754,1528,0,0
1,deepseek-chat-v3.1,field,0.494317,0.383770,1460,0,0
2,deepseek-chat-v3.1,seniority,0.524840,0.390060,1516,0,0
3,deepseek-chat-v3.1,top_k,0.491867,0.386214,1486,0,0
4,deepseek-chat-v3.1,twins,0.387920,0.395124,4155,0,0


In [42]:
# Group: model
# Subgroup: task_name

df_summary = pd.DataFrame()
infra_class = ['model_access', 'model_size', 'model_class']
df_summary = pd.DataFrame()
for c in infra_class:
    tmp = df.query("model.str.contains('gemini') and task_param=='top_100'").groupby([c, 'task_param'])['metric_value'].agg(['mean', 'std', 'count'])
    tmp.loc[:,'group'] = c.split('_')[1]
    tmp.loc[:,'ci_low'] = 0
    tmp.loc[:,'ci_high'] = 0
    tmp.reset_index(inplace=True)
    tmp.rename(columns={c:'subgroup'}, inplace=True)


    df_summary = pd.concat([df_summary, tmp])
df_summary.head(5)


,subgroup,task_param,mean,std,count,group,ci_low,ci_high
0,proprietary,top_100,0.530063,0.369376,553,access,0,0
0,M,top_100,0.531111,0.371507,265,size,0,0
1,S,top_100,0.529099,0.368048,288,size,0,0
0,reasoning,top_100,0.530063,0.369376,553,class,0,0


In [38]:
df.task_param.unique()

array(['1950s', '2000s', 'CM&MP', 'PER', 'early_career', 'senior',
       'top_100', 'top_5', 'famous_female', 'famous_male',
       'fictitious_female', 'fictitious_male', 'movie_female',
       'movie_male', 'politic_female', 'politic_male', 'random_female',
       'random_male'], dtype=object)